In [1]:
#import libs i'll import them in middle of notebook also as per my requirements
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras
import nltk

2025-08-03 16:08:11.929989: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-03 16:08:12.065819: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-03 16:08:12.108265: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-03 16:08:13.655438: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

In [2]:
#loading csv
df=pd.read_csv("Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv")
df.columns

Index(['id', 'dateAdded', 'dateUpdated', 'name', 'asins', 'brand',
       'categories', 'primaryCategories', 'imageURLs', 'keys', 'manufacturer',
       'manufacturerNumber', 'reviews.date', 'reviews.dateSeen',
       'reviews.didPurchase', 'reviews.doRecommend', 'reviews.id',
       'reviews.numHelpful', 'reviews.rating', 'reviews.sourceURLs',
       'reviews.text', 'reviews.title', 'reviews.username', 'sourceURLs'],
      dtype='object')

In [3]:
df.shape

(28332, 24)

In [4]:
df.head(1)

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."


In [5]:
#only 2 cols are imp as we have to perform normal sentiment analysis 
df["reviews.rating"].isna().sum()

0

In [6]:
df['reviews.text'].isna().sum()

0

In [7]:
df=df[['reviews.rating','reviews.text']]

In [8]:
df.isna().sum()

reviews.rating    0
reviews.text      0
dtype: int64

In [9]:
#dropping all the cols except these 2
df=df.dropna()

In [10]:
df.isna().sum()

reviews.rating    0
reviews.text      0
dtype: int64

In [11]:
text=df['reviews.text']

In [12]:
#now nlp part starts
import re
import string 

def clean_text(text):
    text = text.lower()  # lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # remove digits
    text = text.translate(str.maketrans('', '', string.punctuation))  # remove punctuation
    text = text.strip()  # remove leading/trailing whitespace
    return text


In [13]:
text=text.apply(clean_text)
text

0        i order  of them and one of the item is bad qu...
1        bulk is always the less expensive way to go fo...
2        well they are not duracell but for the price i...
3        seem to work as well as name brand batteries a...
4        these batteries are very long lasting the pric...
                               ...                        
28327    i got  of these for my  yr old twins my  yr ol...
28328    i bought this for my niece for a christmas gif...
28329    very nice for light internet browsing keeping ...
28330    this tablet does absolutely everything i want ...
28331    at ninety dollars the expectionations are low ...
Name: reviews.text, Length: 28332, dtype: object

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect=TfidfVectorizer(stop_words='english',max_features=5000)

In [15]:
x=vect.fit_transform(text)

In [16]:
def get_sentiment(rating):
    if rating >= 4:
        return 'Positive'
    elif rating == 3:
        return 'Neutral'
    else:
        return 'Negative'

df['sentiment'] = df['reviews.rating'].apply(get_sentiment)

In [17]:
df['sentiment']=df['sentiment'].map({'Positive':2,'Neutral':1,'Negative':0})

In [18]:
y=df['sentiment']

In [19]:
from imblearn.over_sampling import SMOTE
sm=SMOTE()
x_new , y_new =sm.fit_resample(x,y)

In [20]:
x_new.shape[0]

76635

In [21]:
y_new.shape[0]

76635

In [22]:
df

,reviews.rating,reviews.text,sentiment
0,3,I order 3 of them and one of the item is bad q...,1
1,4,Bulk is always the less expensive way to go fo...,2
2,5,Well they are not Duracell but for the price i...,2
3,5,Seem to work as well as name brand batteries a...,2
4,5,These batteries are very long lasting the pric...,2
...,...,...,...
28327,5,I got 2 of these for my 8 yr old twins. My 11 ...,2
28328,4,I bought this for my niece for a Christmas gif...,2
28329,5,"Very nice for light internet browsing, keeping...",2
28330,5,This Tablet does absolutely everything I want!...,2


In [23]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_new,y_new,test_size=0.2, random_state=42, stratify=y_new)

In [44]:
from sklearn.linear_model import LogisticRegression
modellog = LogisticRegression(max_iter=1000, class_weight='balanced')

In [45]:
modellog.fit(x_train,y_train)

LogisticRegression(class_weight='balanced', max_iter=1000)

In [46]:
y_pred=modellog.predict(x_test)

In [47]:
from sklearn.metrics import accuracy_score, classification_report
accuracy=accuracy_score(y_test,y_pred)
accuracy

0.8874535134077118

In [28]:
print("\n",classification_report(y_test,y_pred))


               precision    recall  f1-score   support

           0       0.90      0.89      0.90      5109
           1       0.83      0.90      0.87      5109
           2       0.93      0.87      0.90      5109

    accuracy                           0.89     15327
   macro avg       0.89      0.89      0.89     15327
weighted avg       0.89      0.89      0.89     15327



In [29]:
df['sentiment'].value_counts()

sentiment
2    25545
0     1581
1     1206
Name: count, dtype: int64

In [48]:
def preprocess(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

# Example input
new_text = "This product is absolutely wonderful!"
clean_text = preprocess(new_text)

# Vectorize like training
vec_text = vect.transform([clean_text])

# Predict
prediction = modellog.predict(vec_text)
print(prediction)  # e.g., ['Positive']



[2]


In [49]:
y_train_pred = modellog.predict(x_train)
y_test_pred = modellog.predict(x_test)

print("Train Accuracy:", accuracy_score(y_train, y_train_pred))
print("Test Accuracy:", accuracy_score(y_test, y_test_pred))

Train Accuracy: 0.9013831800091342
Test Accuracy: 0.8874535134077118


In [50]:
def predict_sentiment(text):
    text_cleaned = preprocess(text)
    vector = vect.transform([text_cleaned])
    prediction = modellog.predict(vector)
    return prediction[0]

In [33]:
t=input("Enter your text:")
if predict_sentiment(t)==2:
    print("Positive")
elif predict_sentiment(t)==3:
    print("Neutral")
else:
    print("Negative")

Enter your text: bad product


Negative


In [34]:
from datasets import load_dataset
dataset = load_dataset("cnn_dailymail", '3.0.0')
sample_data = dataset["train"].select(range(1000))  

# Convert to pandas DataFrame
df = sample_data.to_pandas()

# Preview
print(df.head())

/home/arpit/miniconda3/envs/tf-gpu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


                                             article  \
0  LONDON, England (Reuters) -- Harry Potter star...   
1  Editor's note: In our Behind the Scenes series...   
2  MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...   
3  WASHINGTON (CNN) -- Doctors removed five small...   
4  (CNN)  -- The National Football League has ind...   

                                          highlights  \
0  Harry Potter star Daniel Radcliffe gets £20M f...   
1  Mentally ill inmates in Miami are housed on th...   
2  NEW: "I thought I was going to die," driver sa...   
3  Five small polyps found during procedure; "non...   
4  NEW: NFL chief, Atlanta Falcons owner critical...   

                                         id  
0  42c027e4ff9730fbb3de84c1af0d2c506e41c3e4  
1  ee8871b15c50d0db17b0179a6d2beab35065f1e9  
2  06352019a19ae31e527f37f7571c6dd7f0c5da37  
3  24521a2abb2e1f5e34e6824e0f9e56904a2b0e88  
4  7fe70cc8b12fab2d0a258fababf7d9c6b5e1262a  


In [35]:
from transformers import T5Tokenizer, T5ForConditionalGeneration


tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

def summarize(text):
    input_text = "summarize: " + text
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs, max_length=150, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Original:\n", df['article'][0][:500]) 
print("\nT5 Summary:\n", summarize(df['article'][0]))


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Original:
 LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as s

T5 Summary:
 the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. he will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II" despite his growing fame, he says he is keeping his feet firmly on the ground.


In [36]:
for i in range(10):
    print(f"\n📰 Original Article {i+1}:\n", df['article'][i][:400], "...")
    print("\n📝 T5 Summary:\n", summarize(df['article'][i]))
    print("="*100)



📰 Original Article 1:
 LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his ca ...

📝 T5 Summary:
 the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. he will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II" despite his growing fame, he says he is keeping his feet firmly on the ground.

📰 Original Article 2:
 Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An 

In [37]:
df_small = df.head(10).copy()

# Generate summaries
df_small["generated_summary"] = df_small["article"].apply(summarize)

# View results side by side
print(df_small[["article", "highlights", "generated_summary"]].head())


                                             article  \
0  LONDON, England (Reuters) -- Harry Potter star...   
1  Editor's note: In our Behind the Scenes series...   
2  MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...   
3  WASHINGTON (CNN) -- Doctors removed five small...   
4  (CNN)  -- The National Football League has ind...   

                                          highlights  \
0  Harry Potter star Daniel Radcliffe gets £20M f...   
1  Mentally ill inmates in Miami are housed on th...   
2  NEW: "I thought I was going to die," driver sa...   
3  Five small polyps found during procedure; "non...   
4  NEW: NFL chief, Atlanta Falcons owner critical...   

                                   generated_summary  
0  the young actor says he has no plans to fritte...  
1  inmates with most severe mental illnesses are ...  
2  survivor: "i probably had a 30-, 35-foot free ...  
3  the polyps were removed and sent to the nation...  
4  new: "your admitted conduct was not only illeg..

In [38]:
import spacy

# Load English NER model
nlp = spacy.load("en_core_web_sm")

# NER function
def extract_entities(text):
    doc = nlp(text)
    return [(ent.text, ent.label_) for ent in doc.ents]
    
text = df_small["article"][0]  # Or any article
entities = extract_entities(text)

for entity, label in entities:
    print(f"{entity} → {label}")


LONDON → GPE
England → GPE
Reuters → ORG
Harry Potter → PERSON
Daniel Radcliffe → PERSON
£20 million → MONEY
$41.1 million → MONEY
18 → CARDINAL
Monday → DATE
Daniel Radcliffe → PERSON
Harry Potter → PERSON
Harry Potter → PERSON
the Order of the Phoenix → PERSON
18 → DATE
Australian → NORP
earlier this month → DATE
about 10 pounds → QUANTITY
18 → DATE
Radcliffe → PERSON
Hostel: Part II → WORK_OF_ART
six → CARDINAL
one → CARDINAL
UK → GPE
Radcliffe → PERSON
first → ORDINAL
five → CARDINAL
last month → DATE
Harry Potter → PERSON
the Order of the Phoenix → PERSON
Atlantic → LOC
two → CARDINAL
Potter → PERSON
Potter → PERSON
Londoner → PERSON
My Boy Jack → WORK_OF_ART
Rudyard Kipling → PERSON
later this year → DATE
December Boys → WORK_OF_ART
Australian → NORP
about four → CARDINAL
Earlier this year → DATE
Peter Shaffer's → PERSON
Reuters → ORG
2007 → DATE
Reuters → ORG


In [39]:
df_small["named_entities"] = df_small["article"].apply(extract_entities)

In [51]:
import pickle

# ✅ Save the logistic regression sentiment model
with open("sentiment_model.pkl", "wb") as f:
    pickle.dump(modellog, f)

# ✅ Save the TF-IDF vectorizer
with open("vectorizer.pkl", "wb") as f:
    pickle.dump(vect, f)
